In [50]:
import os
import cv2
import numpy as np
import csv
import glob as glob
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
from lxml import etree
import json
from collections import defaultdict
import cv2
import matplotlib.pyplot as plt
import math
import scipy.io
import pandas as pd
from scipy.io import loadmat
from ast import literal_eval

In [51]:
# source = "MW-R"
OUTPUT_DIR ="output"
INPUT_DIR = "input"

def write_xml(xml_str, xml_path):
    # remove blank text before prettifying the xml
    parser = etree.XMLParser(remove_blank_text=True)
    root = etree.fromstring(xml_str, parser)
    # prettify
    xml_str = etree.tostring(root, pretty_print=True)
    # save to file
    with open(xml_path, 'wb') as temp_xml:
        temp_xml.write(xml_str)


def create_PASCAL_VOC_xml(xml_path, abs_path, folder_name, image_name, img_height, img_width, depth):
    # By: Jatin Kumar Mandav
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = folder_name
    ET.SubElement(annotation, 'filename').text = image_name
    ET.SubElement(annotation, 'path').text = abs_path
    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = img_width
    ET.SubElement(size, 'height').text = img_height
    ET.SubElement(size, 'depth').text = depth
    ET.SubElement(annotation, 'segmented').text = '0'
    print(xml_path)
    xml_str = ET.tostring(annotation)
    write_xml(xml_str, xml_path)

def voc_format(class_name, point_1, point_2):
    # Order: class_name xmin ymin xmax ymax
    xmin, ymin = min(point_1[0], point_2[0]), min(point_1[1], point_2[1])
    xmax, ymax = max(point_1[0], point_2[0]), max(point_1[1], point_2[1])
    items = map(str, [class_name, xmin, ymin, xmax, ymax])
    return items
    
def append_bb(ann_path, line, extension):
    if '.txt' in extension:
        with open(ann_path, 'a') as myfile:
            myfile.write(line + '\n') # append line
    if '.xml' in extension:

        class_name, xmin, ymin, xmax, ymax = line

        tree = ET.parse(ann_path)
        annotation = tree.getroot()

        obj = ET.SubElement(annotation, 'object')
        ET.SubElement(obj, 'name').text = class_name
        ET.SubElement(obj, 'pose').text = 'Unspecified'
        ET.SubElement(obj, 'truncated').text = '0'
        ET.SubElement(obj, 'difficult').text = '0'

        bbox = ET.SubElement(obj, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = xmin
        ET.SubElement(bbox, 'ymin').text = ymin
        ET.SubElement(bbox, 'xmax').text = xmax
        ET.SubElement(bbox, 'ymax').text = ymax

        xml_str = ET.tostring(annotation)
        write_xml(xml_str, ann_path)


def get_annotation_paths(img_path, annotation_formats):
    annotation_paths = []
    for ann_dir, ann_ext in annotation_formats.items():
        new_path = os.path.join(OUTPUT_DIR, ann_dir)
        new_path = img_path.replace(INPUT_DIR, new_path, 1)
        pre_path, img_ext = os.path.splitext(new_path)
        new_path = new_path.replace(img_ext, ann_ext, 1)
        annotation_paths.append(new_path)
    return annotation_paths

def save_bounding_box(annotation_paths, class_index, point_1, point_2, width, height):
    for ann_path in annotation_paths:
        # if '.txt' in ann_path:
        #     line = yolo_format("person", point_1, point_2, width, height)
        #     append_bb(ann_path, line, '.txt')
        print(ann_path)
        if '.xml' in ann_path:
            line = voc_format("head", point_1, point_2)
            append_bb(ann_path, line, '.xml')

annotation_formats = {'PASCAL_VOC' : '.xml', 'YOLO_darknet' : '.txt'}
annotation_formats = {'PASCAL_VOC' : '.xml'}
desired_img_format = '.jpg'

In [10]:
# vid_folder = './Meeting'
folder_ = r'D:\Work\JHU++\WIKI'
folder = os.path.normpath(folder_)
# annotation_paths = folder_ +"\\annotations" 
annotation_paths= os.path.join(os.path.normpath(folder_), "annotations")
annotation_paths =folder
annotations = glob.glob(annotation_paths+"/*.mat")

In [11]:
annotations

['D:\\Work\\JHU++\\WIKI\\wiki.mat']

In [12]:
def drawbbox(im, x, y, w, h, color=(255,0,0), linewidth=1,annotation_paths=annotation_paths):
#     left = int(int(x) - int(w) / 2)
#     top = int(int(y) - int(h) / 2)
    cv2.rectangle(im ,(int(x), int(y)),(int(w), int(h)), (0,255,0), 1)
#     save_bounding_box(annotation_paths, 0, (int(x), int(y)),(int(w), int(h)), im.shape[0], im.shape[1])

In [13]:
annotations

['D:\\Work\\JHU++\\WIKI\\wiki.mat']

In [14]:
depth=str(3)

for annotation in annotations:
    print(annotation)
    ann_data = scipy.io.loadmat(annotations[0])
    print(ann_data['imdb'])
#     print(ann_data["wiki"][0])
#     print(ann_data["bbox"])
#     #ann_data = json.load(open(annotation, 'r'))
#     # Print the information of the first image
#     #print(ann_data["annBoxes"])
#     boxes = ann_data["bbox"]
#     annotation_ = annotation
#     annotation_ = annotation_.replace("BBOX", "IMG")
#     frame_path = r"D:\Work\JHU++\bbox\IMG_0000.png"
#     frame_path = annotation_[:-4]+".png"
#     print(frame_path)
#     if not os.path.exists(frame_path):
#         continue
#     im = plt.imread(frame_path)
#     #print(Lines)
#     annotation_paths = get_annotation_paths(frame_path, annotation_formats)
#     abs_path = os.path.abspath(frame_path)
#     folder_name = os.path.dirname(frame_path)
#     image_name = os.path.basename(frame_path)
#     print(image_name)
#     #cout << "Width : " << im.cols << endl;
#     #cout << "Height: " << im.rows << endl;
#     img_height = str(im.shape[0])
#     img_width = str(im.shape[1])
#     print('width: ', im.shape[1])
#     print('height:', im.shape[0])
#     for ann_path_ in annotation_paths:
#         if not os.path.isfile(ann_path_):
#             create_PASCAL_VOC_xml(ann_path_, abs_path, folder_name, image_name, img_height, img_width, depth)
#     for line in boxes:
#         print(line)
#         # label=line.rstrip("\n").replace(" ",",").split(",")
#         x,y,w,h= int(line[0]),int(line[1]),int(line[2]),int(line[3])
#         drawbbox(im,x,y,w,h,annotation_paths=annotation_paths)
#     plt.figure(figsize = (8,8))
#     plt.imshow(im)
#     plt.show()
    

D:\Work\person\imdb\imdb.mat
[[(array([[693726, 693726, 693726, ..., 726831, 726831, 726831]]), array([[1968, 1970, 1968, ..., 2011, 2011, 2011]], dtype=uint16), array([[array(['01/nm0000001_rm124825600_1899-5-10_1968.jpg'], dtype='<U43'),
        array(['01/nm0000001_rm3343756032_1899-5-10_1970.jpg'], dtype='<U44'),
        array(['01/nm0000001_rm577153792_1899-5-10_1968.jpg'], dtype='<U43'),
        ...,
        array(['08/nm3994408_rm926592512_1989-12-29_2011.jpg'], dtype='<U44'),
        array(['08/nm3994408_rm943369728_1989-12-29_2011.jpg'], dtype='<U44'),
        array(['08/nm3994408_rm976924160_1989-12-29_2011.jpg'], dtype='<U44')]],
      dtype=object), array([[1., 1., 1., ..., 0., 0., 0.]]), array([[array(['Fred Astaire'], dtype='<U12'),
        array(['Fred Astaire'], dtype='<U12'),
        array(['Fred Astaire'], dtype='<U12'), ...,
        array(['Jane Levy'], dtype='<U9'),
        array(['Jane Levy'], dtype='<U9'),
        array(['Jane Levy'], dtype='<U9')]], dtype=object)

In [26]:
ann_data["imdb"][0][0][0][0]

array([693726, 693726, 693726, ..., 726831, 726831, 726831])

In [14]:
def get_names(x):
    if len(x)>0:
        return x[0]
    else:
        return ''

In [36]:
def extract_mat(mat_file, dataset):
    mat = loadmat(mat_file)  # load mat-file
    mdata = mat[dataset]  # variable in mat file
    mdtype = mdata.dtype
    ndata = {n: mdata[n][0, 0] for n in mdtype.names}
    columns = [n for n, v in ndata.items()]# if v.size == ndata['numIntervals']]

    dob = mdata['dob'][0,0][0]
    photo_taken = mdata['photo_taken'][0,0][0]
    full_path = [mdata['full_path'][0,0][0][n][0] for n in range(len(mdata['full_path'][0,0][0]))]
    gender = mdata['gender'][0,0][0]
    name = np.array(list(map(get_names, mdata['name'][0,0][0])))
    face_location = mdata['face_location'][0,0][0]
    print(face_location)
    faces=[]
    for f in face_location:
        faces.append(list([f[0][0],f[0][1],f[0][2],f[0][3]]))
    face_score = mdata['face_score'][0,0][0]
    second_face_score = mdata['second_face_score'][0,0][0]
    #celeb_id = mdata['celeb_id'][0,0][0]
    metadf = pd.DataFrame({"dob": dob, "photo_taken":photo_taken, "full_path":full_path, "gender":gender, "name":name, "face_location":faces, "face_score":face_score, "second_face_score":second_face_score})#index=celeb_id)
    metadf['full_path'] = metadf['full_path'].apply(lambda x: x.split('/')[1])
    print(dataset)
    
    metadf.to_csv(f"{dataset}.csv")
    return face_location
    

In [37]:
face_location= extract_mat(r'D:\Work\JHU++\WIKI\wiki.mat',"wiki")

[array([[111.29109473, 111.29109473, 252.66993082, 252.66993082]])
 array([[252.4833023 , 126.68165115, 354.53192596, 228.73027481]])
 array([[113.52, 169.84, 366.08, 422.4 ]]) ...
 array([[169.88839786,  74.31669472, 235.2534231 , 139.68171997]])
 array([[1, 1, 1, 1]], dtype=uint8)
 array([[ 92.72633235,  62.0435549 , 230.12083087, 199.43805342]])]
wiki


In [55]:
type(face_location)

numpy.ndarray

In [78]:
# type(face_location[0][0])
# face_location[0]
# face_location[1]

numpy.ndarray

[array([[111.29109473, 111.29109473, 252.66993082, 252.66993082]])
 array([[252.4833023 , 126.68165115, 354.53192596, 228.73027481]])
 array([[113.52, 169.84, 366.08, 422.4 ]]) ...
 array([[169.88839786,  74.31669472, 235.2534231 , 139.68171997]])
 array([[1, 1, 1, 1]], dtype=uint8)
 array([[ 92.72633235,  62.0435549 , 230.12083087, 199.43805342]])]
wiki


In [48]:
for face in face_location:
    print(face[0])

111.29109473290997
252.48330229530742
113.52
1
171.61031405173117
274.76563240288175
79.35580188693945
152.9073113210374
1
1
107.58644628233198
240.23999999999998
75.25800000000001
1
1
113.52
221.87542481104154
130.93069968577643
1
126.47169334270646
54.588957565949265
87.18794319984659
126.36258903238176
148.22336338911953
117.08654109838353
1
86.24515702586558
105.97683616670597
138.11816488834953
1
1
129.8283493174264
159.11098870136559
38.94626472790037
122.22159337927492
130.269382706866
249.85666584231726
156.21309994205467
183.92
1
104.76830673359814
104.8870800393785
89.9813914361322
162.12
1
56.156375106748115
1
101.25014680821972
36.627197596249545
1
48.90889496679273
146.44305554522424
138.11816488834953
214.57028154520884
205.6
181.4670637253251
245.45446244393372
140.26488870749222
69.03230987056897
160.1770156657054
1
1
124.32569680891355
1
1
123.45351931209424
1
1
96.2217031357533
62.98167381711889
1
169.69701257893047
1
1
1
190.39312702598662
0.8799999999999999
112.3520

1
1
211.07367233341193
182.56899344768536
70.75174564077851
195.30073359182848
168.08804245314116
179.50575530258996
151.78570352511005
114.09570986850775
209.95733306123833
200.00126958213175
113.52
1
145
1
1
1
1
1
1
1
1
1
217.1691492551213
130.49507031206528
119.62304834927758
55.79183898752064
169.83999999999997
81.78461980628043
1
1
188.106
349.34222176873055
203.62224793643077
91.42342211506603
151.32021977110426
1
1
156.91094380361636
1
1
1
1
134.64
1
1
127.77712035106016
203.17091047947815
195.94057512549648
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
107.58644628233198
31.04871387294425
1
80.5285078328527
189.504
104.8870800393785
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
28.16680713932678
1
174.22513339896418
162.61672838358248
98.93881544967539
115.24513740695794
1
130.269382706866
156.89062005906774
1
1
90.40461701782878
1
177.45143178283695
1
101.93820754735827
1
1
1
114.76218166976636
128.92773553879837
141.67999999999998
1

1
182.01933598375615
149.50937106945878
107.58644628233198
212.126
193.06477199451533
115.24513740695794
200.3679755345098
115.24513740695794
125.56196808031775
132.8100504956584
85.36
1
123.45351931209424
1
30.73284395422085
267.302
0.54
167.149290773757
190.35000000000002
79.15105235974032
1
113.52
268.40999999999997
381.03999999999996
158.35331017399295
112.3520283020941
1
194.79042253136876
91.30966799187807
1
1
193
172.44626179277228
79.39425934793945
156.70339280599768
129.14984799634354
1
144.10509788764284
169.89701257893046
122.2573253208318
1
131.30960950394757
1
179.208587920045
183.92
1
1
120.55999999999999
1
138.11816488834953
1
171.20862231591292
1
138.402
125.57444378954013
1
107.58644628233198
1
143.2568543634726
1
195.30073359182848
115.24513740695794
124.36830288418096
197.99999999999997
1
127.6
1
1
114.44385561877563
57.11467982776716
218.02320318050738
191.55964673886058
65.574691353433
112.3520283020941
138.11816488834953
128.538
105.59772453857046
1
191.5596467388

91.42342211506603
200.00126958213175
1
146.44305554522424
151.78570352511005
150.2209521865988
186.4
70.21805335958567
183.67932819968738
74.42411158725653
170.12125511037777
155.3353786052262
115.24513740695794
156.89062005906774
136.19071828445084
25.934551886839568
115.24513740695794
51.835140330284325
57.199999999999996
184.72861983702268
1
1
166.8762738776211
1
131.85
85.36
126.68165114765371
46.15171105753302
128.4550527756009
1
1
1
59.959456141617316
561.334
1
166.8762738776211
156.05668979172475
155.76
136.50820435933161
123.45351931209424
159.3866071963711
1
148.03338943962046
141.67999999999998
138.11816488834953
115.24513740695794
71.27999999999999
188.04745353340337
65.574691353433
167.2
195.8052750738347
120.29623741698181
1
39.67017122813967
149.55516509461668
1
35.54902667979283
99.33
87.53976823948501
123.45351931209424
1
1
65.574691353433
179.14310416603917
204.60269975889858
172.4837540058069
1
1
175.51342908592304
281.9525191147814
160.1770156657054
69.59350160232276

141.67999999999998
188.55
137.12299213861795
1
0.33999999999999997
1
289
1
1
170.133079144812
183.86421985113267
86.24515702586558
79.37591330394413
61.24228141004402
88.98551008496263
181.96684423013207
160.93966942349795
89.02003366377228
162.53672838358253
158.58648490814926
212.07999999999998
65.574691353433
1
203.05091047947812
181.96684423013207
359.6488058632181
140.40843876959823
1
1
100.96972248446362
65.574691353433
1
84.7608971562815
161.36916775584453
1
261.48160831859667
162.08022836152676
180.142334375067
325.3272625789866
1
120.35276174927905
138.11816488834953
1
106.10754826723299
14.696246069209089
1
128.92773553879837
254.14551886839567
1
1
214.9065064356554
158.52525425005896
52.863164538599335
1
85.36
1
200.00126958213175
128.92773553879837
1
1
174.42155905387656
1
1
77.35701854540264
1
89.09239776083396
84.15975604169793
106.51399246757704
1
258.22563100871037
127.51353518862929
46.840613581567865
77.39999999999999
156.89062005906774
66.07895567324917
126.136107932

...

244.8880685014962
1
176.17699009553698
105.97683616670597
138.96
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
245.7
192.95160330819755
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
91.42342211506603
138.11816488834953
113.52
1
1
97.80586025666544
149.50937106945878
119.14
199.721154646736
205.20891630699361
180.18
60.16181598840193
206.11082859255714
136.60820435933164
123.45351931209424
56.501999999999995
1
135.91761006314437
147.83338943962048
104.39727633488869
118.4625683894415
138.11816488834953
85.14
61.24228141004402
149.50937106945878
187.94295212047663
307.66076886849555
114.09570986850775
166.8762738776211
181.96684423013207
123.64800000000001
101.08742489171348
1
1
112.23047210190387
1
140.22003537761762
1
115.67999999999999
1
95.45515139147199
151.78570352511005
85.44715879744693
1
1
178.79040122194075
162.725286369478
124.06830288418097
53.42841808335298
1
21.17805048069683
1
104.8870800393785
1
151.78570352511005
149.50

70.21805335958567
132.25104520838246
0.6
175.6021499595493
44.51001683188614
1
138.11816488834953
1
302.81978105662944
105.97683616670597
113.55433670932669
1
1
1
1
76.11063688336156
247.2288826689832
49.90940772483769
171.61031405173117
1
103.89025240348414
127.49
155.76
125.39999999999999
116.1
130.269382706866
1
1
1
129.7450220544303
1
1
207.20489860844847
43.56257851293279
197.99999999999997
169.83999999999997
177.54612144711683
126.90761833366507
105.97683616670597
181.96684423013207
1
74.42411158725653
1
175.51342908592304
74.49600000000001
1
112.43047210190386
68.35880503157219
165.42440384557463
40.34491735587449
180.20084147483107
135.71087124330367
64.05676759431465
145.38814972922071
206.1765864565455
223.92888020754896
1
1
128.0611757486577
1
61.24228141004402
243.56318675854985
97.80586025666544
327.9053127056494
1
158.52525425005896
181.96684423013207
241.07725560812966
140.26488870749222
211.13774689865724
105.97683616670597
1
171.21425798016972
1
127.2735351886293
148.2

70.21805335958567
174.42155905387656
108.60964868337732
1
1
227.3114197370155
101.9662778073293
65.532
1
165.15257640923434
150.26902479526478
276.7591949376031
321.5684967880837
89.57155208301958
68.35880503157219
147.96822317451307
125.86906563829756
1
219.11999999999998
98.93881544967539
78.40468820533235
113.098
1
1
115.24513740695794
184.79946329173544
1
123.99244347931378
1
204.64675298172568
57.617999999999995
1
1
40.704253916426346
145.16694575498553
61.61769196169922
112.3520283020941
76.33285176255502
36.60618184282177
75.19468553472939
75.658
175.56
123.45351931209424
182.364
75.19468553472939
150.28527864588918
204.60269975889858
1
151.78570352511005
1
111.6062819965641
127.51353518862929
257.0780379757425
140.26488870749222
85.36
212.07999999999998
168.08804245314116
1
154.66643558619273
1
95.7
169.83999999999997
139.55610671917134
155.76
92.37210992556635
72.72034831044576
123.45351931209424
1
1
38.0373427673647
94.0334919068715
160.9911923697411
103.43603433375155
158.52

1
111.96205834515338
42.08270514980633
142.4715808087965
1
189.51212940638757
105.97683616670597
1
18.214513339896417
147.96822317451307
1
106.58404039652673
1
1
85.36
105.8396926125394
1
1
136.71051351427997
126.60307544026227
1
142.90854831680812
1
156.30525157261602
1
125.39734763423778
1
61.24228141004402
1
49.90940772483769
129.2453396354647
117.20703230799855
30.59281081930962
70.0677751573615
151.78570352511005
66.85832366142269
1
102.37894059892616
0.876
1
132.25104520838246
1
78.4762300501986
137.45061974113793
156.89062005906774
60.06532286668826
155.39954867714118
133.17
26.419390282482404
1
1
1
184.3731005402192
1
53.42841808335298
16.786238316672744
1
155.76
85.36
113
1
37.05775471704898
1
79.13
115.71000000000001
141.67999999999998
76.92775428658989
67.39404357709917
33.402
1
1
1
156.01864763795206
51.854169609147355
105.97683616670597
1
141.67999999999998
106.0998630278685
218.31518058417012
1
1
160.9911923697411
225
328.10908313740146
89.30265052681094
80.5
1
74.4241115

86.24515702586558
140.26488870749222
243.56318675854985
1
100.92
120.35276174927905
309.44587099878646
182.89239006891236
1
51.06910377367914
46.32219339631123
1
28.237400640929103
136.69513317259904
123.45351931209424
85.36
189.6321294063876
212.07999999999998
113.52
81.54181564474666
94.3675063715938
200.41381792209117
146.31300058989333
1
125.92951443064833
22.131843257135152
137.98700320464553
197.05741458264308
115.24513740695794
159.72898584935248
1
113.52
158.37395300813478
1
117.9643450752979
149.50937106945878
1
171.61031405173117
1
1
138.11816488834953
170.42891646574884
27.56
1
40.704253916426346
191.55964673886058
26.43084315193358
178.79040122194075
32.31729698162985
155.76
121.60456282008805
123.45351931209424
1
1
80.82661636991065
150.26902479526478
41.629350596345134
180.08914262391858
130.88885004922315
76.50377903500359
127.1916039578878
1
217.66154231894808
128.92773553879837
156.89062005906774
1
155.66935954782468
169.83999999999997
65.574691353433
70.83999999999999

In [39]:
# metadf["gender"]

NameError: name 'metadf' is not defined

In [39]:
meta=pd.read_csv("D:\Work\JHU++\wiki.csv" )

In [42]:
meta["face_location"][0]

'[111.29109473290997, 111.29109473290997, 252.66993081807996, 252.66993081807996]'

In [46]:
meta["face_location"] = meta["face_location"].apply(literal_eval)
meta["face_location"]

[111.29109473290997,
 111.29109473290997,
 252.66993081807996,
 252.66993081807996]

In [45]:
df[0]

[111.29109473290997,
 111.29109473290997,
 252.66993081807996,
 252.66993081807996]

In [69]:
meta["face_location"][0]

'[[111.29109473 111.29109473 252.66993082 252.66993082]]'

In [59]:
aa=np.array(meta["face_location"][0])

In [65]:
aa

array('[[111.29109473 111.29109473 252.66993082 252.66993082]]',
      dtype='<U55')

ValueError: invalid literal for int() with base 10: '[[111.29109473'